In [53]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from statistics import *
import heartpy as hp
import copy
pd.options.display.max_rows = 450

patPWV = pd.read_csv("PWVbyPatient.csv", header=None)

In [54]:
#Helper Methods

def segmentProcess1(data, sample_rate=240.0):
    # Change values ouside possible range to min and max pulse value
    data = [0 if i <= 0 else (550 if i > 550 else i) for i in data]
    data = hp.filter_signal(data, cutoff=15, sample_rate=sample_rate, order=4, filtertype='lowpass')
    data = hp.filter_signal(data, cutoff=.01, sample_rate=sample_rate, order=4, filtertype='highpass')
    wd, m = hp.process(hrdata=data, sample_rate=sample_rate, bpmmin=0, bpmmax=550)
    return wd, m

def segmentProcess2(peakList, sample_rate=240.0):
    test = []
    # test.append(0)
    try:
        peakList.insert(0, 0)
    except:
        peakList = np.insert(peakList, 0, 0)
    
    for i in range(len(peakList)-2):
        a = (peakList[i] + peakList[i+1])/2
        b = (peakList[i+1] + peakList[i+2])/2
        interval = .15*(b-a)
        a += interval
        b -= interval
        test.append(round(a))
        # test.append(round(b))
    return test

In [55]:
iter = 20
wds = []
rngs = []
df = pd.DataFrame()

for d in range(iter):
    if(d+1 < 10):
        fileStr = '0' + str(d+1)
    else:
        fileStr = str(d+1)
    data = hp.get_data('Data/Raw Data/Multiple Cath/X0' + fileStr + '.txt', delim = ' ', column_name = 'AO')
    wd, _ = segmentProcess1(data)
    df1 = pd.DataFrame(wd['hr'])
    df = pd.concat([df,df1], axis=1)
    wds.append(wd['peaklist'].copy())
for i in range(iter):
    if(i+1 < 10):
        fileStr = '0' + str(i+1)
    else:
        fileStr = str(i+1)
    rngs.append(segmentProcess2(wds[i]))

/home/anrath/miniconda3/envs/biopeaks/lib/python3.7/site-packages/numpy/ma/core.py:5240: RuntimeWarning: Mean of empty slice.
  dtype=dtype, **kwargs)[()]
/home/anrath/miniconda3/envs/biopeaks/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3703: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)


In [57]:
def calcStats(wave):
    #calculate metrics for each indexed wave saved in individual metric arrays
    #remove outliers, save only mean for each metric
    #append metric means to new df

    #finding maximum, defining region for dic notch
    sysMaxi = wave.idxmax()
    reg = wave[int(round(sysMaxi*1.05)):int(round(sysMaxi+(len(wave)*.28)))]
    diff = reg.diff()

    NP = [0,0]


    #find dic notch
    ser = [0,0]
    counter = 0
    for index,value in diff.items():
        if value < 1:
            counter = counter + 1
        if value >= 0:
            if counter > 0 and counter > ser[1]:
                ser[0] = index
                ser[1] = counter
            counter = 0



    #if no dic notch found:
    try:
        if ser[0] == 0 or ser[1] < len(reg)*.05:
            #estimate diastolic pressure as flattest point -> highest diff
            #range for diastolic pressure:
            diaReg = diff.tail(n=round(len(diff)*.9))
            diaP = diaReg.idxmax()
            dicN = round(mean([diaReg.idxmax(),diaReg.idxmin()]))
            #print("NO DIC NOTCH FOUND, estimating...")
        else:
            dicN = ser[0]
            if round(2*sysMaxi) > dicN+1:
                diaP = wave[dicN+1:round(2*sysMaxi)].idxmax()
            else:
                diaP = dicN+1
    except:
        return


    NP[0] = dicN
    NP[1] = diaP

    #saving dic notch and diastolic pressure for functions
    dicNotch = NP[0]
    diaP = NP[1]

    #beginning and end of wave
    beg = 0
    end = wave.last_valid_index()
    sysMaxi = wave.idxmax()


    #add calculated metrics to lists
    pp_pres.append(np.sum(wave))
    avg_sys_rise.append(wave[beg:sysMaxi].mean())
    sys_rise_area.append(sum(wave[beg:sysMaxi]))
    t_sys_rise.append(sysMaxi)
    avg_dec.append(wave[sysMaxi:end].mean())
    t_dec.append(end - sysMaxi)
    dec_area.append(np.sum(wave[sysMaxi:end]))
    avg_sys.append(wave[beg:dicNotch].mean())
    slope_sys.append((wave[dicNotch] - wave[beg]) / dicNotch)
    sys_area.append(sum(wave[beg:dicNotch]))
    t_sys.append(dicNotch)
    avg_sys_dec.append(wave[sysMaxi:dicNotch].mean())
    dn_sys.append(wave[sysMaxi] - wave[dicNotch])
    sys_dec_area.append(np.sum(wave[sysMaxi:dicNotch]))
    t_sys_dec.append(dicNotch - sysMaxi)
    avg_sys_dec_nodia.append(wave[sysMaxi:dicNotch].mean() - wave[diaP])
    avg_sys_nodia.append(wave[beg:dicNotch].mean() - wave[diaP])
    avg_sys_rise_nodia.append(wave[beg:sysMaxi].mean() - wave[diaP])
    avg_dec_nodia.append(wave[sysMaxi:end].mean() - wave[diaP])
    slope_dia.append((wave[end] - wave[dicNotch]) / (end - dicNotch))
    t_dia.append(end - dicNotch)
    avg_dia.append(wave[dicNotch:end].mean())
    dn_dia.append(wave[diaP] - wave[dicNotch])

In [58]:
metrics = []

for j in range(len(wds)):
    pp_pres = []
    avg_sys_rise = []
    sys_rise_area = []
    t_sys_rise = []
    avg_dec = []
    t_dec = []
    dec_area = []
    avg_sys = []
    slope_sys = []
    sys_area = []
    t_sys = []
    avg_sys_dec = []
    dn_sys = []
    sys_dec_area = []
    t_sys_dec = []
    avg_sys_dec_nodia = []
    avg_sys_nodia = []
    avg_sys_rise_nodia = []
    avg_dec_nodia = []
    slope_dia = []
    t_dia = []
    avg_dia = []
    dn_dia = []

    for i in range(len(rngs[j])-1):
        # print("{a} {b}".format(a=j, b=i))
        lowerB = rngs[j][i]
        upperB = rngs[j][i+1]
        wave = df.iloc[lowerB:upperB,j].reset_index(drop=True)
        calcStats(wave)
    metrics.append([pp_pres,avg_sys_rise,sys_rise_area,t_sys_rise,avg_dec,t_dec,dec_area,avg_sys,slope_sys,sys_area,t_sys,avg_sys_dec,dn_sys,sys_dec_area,t_sys_dec,avg_sys_dec_nodia,avg_sys_nodia,avg_sys_rise_nodia,avg_dec_nodia,slope_dia,t_dia,avg_dia,dn_dia,avg_sys_nodia])

In [59]:
# Removing the Outliers
for pat in range(len(m1)):
    for feat in range(len(m1[pat])):
        Q1 = np.percentile(m1[pat][feat], 25, interpolation = 'midpoint') 
        Q3 = np.percentile(m1[pat][feat], 75, interpolation = 'midpoint') 
        IQR = Q3 - Q1 
        
        [value for value in m1[pat][feat] if value < (Q3+1.5*IQR) and value > (Q1-1.5*IQR)]

### Metrics is a list of patients. Each patient list has a list of features. Each list of features has values for each wave calculated.
- No outlier math was done.

# Testing

In [37]:
m1 = metrics
p1 = m1[0][1] # pp_pres
print(p1)

[21.918096775290028, 7.320903941892177, 8.46308470901489, 5.886691434513426, 28.62240632592815, 11.419684883205445, 7.528060480457145, -9.20217380599224, 23.539330641019735, 3.4609568363302134, 4.363186648136857, -5.631543200661618, 20.294285134952975, 1.1676147553253682, 4.695212081526027, 0.5771139979413021, -10.161681407796197, 16.526089982968045, -5.983495871012943, -9.222296640601012, -24.99628435419805, -8.360229012944043, -6.809776554023589, -4.965500364107032, nan, -17.457002387674933, 9.070285662998433, 5.361254108680158, -7.72426788982375, -6.239394916610785, nan, -7.908052122059977, -7.6172231726880435, -27.711861809384732, -8.010705938133817, 24.57672037914557, -11.769024445611299, 11.148061312446655, nan, -14.26590401520075, -10.220851383173592, -11.487456802860603, -11.052803922385952, -10.825497895721863, -10.719582605942664, -10.760705916712629, -10.475378494513576, -10.372447918270286, -10.146022442174361, -9.989196398165689, -9.583035734750753, -9.383038649998532, -9.

In [49]:
for pat in range(len(m1)):
    for feat in range(len(m1[pat])):
        Q1 = np.percentile(m1[pat][feat], 25, interpolation = 'midpoint') 
        Q3 = np.percentile(m1[pat][feat], 75, interpolation = 'midpoint') 
        IQR = Q3 - Q1 
        
        # Removing the Outliers
        [value for value in m1[pat][feat] if value < (Q3+1.5*IQR) and value > (Q1-1.5*IQR)]

In [65]:
for i in range(len(rngs[0])-1):
    lowerB = rngs[0][i]
    upperB = rngs[0][i+1]

    # wave = wds[0]['hr'][lowerB:upperB]
    # wave = pd.DataFrame(wave)
    # calcStats(wave)
    wave = patPWV.iloc[lowerB:upperB,0].reset_index(drop=True)
    sysMaxi = wave.idxmax()
    reg = wave[int(round(sysMaxi*1.05)):int(round(sysMaxi+(len(wave)*.28)))]
    diff = reg.diff()

    NP = [0,0]

    #find dic notch
    ser = [0,0]
    counter = 0
    for index,value in diff.items():
        if value < 1:
            counter = counter + 1
        if value >= 0:
            if counter > 0 and counter > ser[1]:
                ser[0] = index
                ser[1] = counter
            counter = 0
    
    

    
    #if no dic notch found:
    try:
        if ser[0] == 0 or ser[1] < len(reg)*.05:
            #estimate diastolic pressure as flattest point -> highest diff
            #range for diastolic pressure:
            diaReg = diff.tail(n=round(len(diff)*.9))
            diaP = diaReg.idxmax()
            dicN = round(mean([diaReg.idxmax(),diaReg.idxmin()]))
            #print("NO DIC NOTCH FOUND, estimating...")
        else:
            dicN = ser[0]
            if round(2*sysMaxi) > dicN+1:
                diaP = wave[dicN+1:round(2*sysMaxi)].idxmax()
            else:
                diaP = dicN+1
    except:
        continue

    NP[0] = dicN
    NP[1] = diaP


    #saving dic notch and diastolic pressure for functions
    dicNotch = NP[0]
    diaP = NP[1]
    
    #beginning and end of wave
    beg = 0
    end = wave.last_valid_index()
    sysMaxi = wave.idxmax()
    

    #add calculated metrics to lists
    pp_pres.append(np.sum(wave))
    avg_sys_rise.append(wave[beg:sysMaxi].mean())
    sys_rise_area.append(sum(wave[beg:sysMaxi]))
    t_sys_rise.append(sysMaxi)
    avg_dec.append(wave[sysMaxi:end].mean())
    t_dec.append(end - sysMaxi)
    dec_area.append(np.sum(wave[sysMaxi:end]))
    avg_sys.append(wave[beg:dicNotch].mean())
    slope_sys.append((wave[dicNotch] - wave[beg]) / dicNotch)
    sys_area.append(sum(wave[beg:dicNotch]))
    t_sys.append(dicNotch)
    avg_sys_dec.append(wave[sysMaxi:dicNotch].mean())
    dn_sys.append(wave[sysMaxi] - wave[dicNotch])
    sys_dec_area.append(np.sum(wave[sysMaxi:dicNotch]))
    t_sys_dec.append(dicNotch - sysMaxi)
    avg_sys_dec_nodia.append(avg_sys_dec[i] - wave[diaP])
    avg_sys_nodia.append(avg_sys[i] - wave[diaP])
    avg_sys_rise_nodia.append(avg_sys_rise[i] - wave[diaP])
    avg_dec_nodia.append(avg_dec[i] - wave[diaP])
    slope_dia.append((wave[end] - wave[dicNotch]) / (end - dicNotch))
    t_dia.append(end - dicNotch)
    avg_dia.append(wave[dicNotch:end].mean())
    dn_dia.append(wave[diaP] - wave[dicNotch])

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

In [ ]:
#remove outliers and calculate mean per metic in each list    
    
pp_pres = pd.Series(pp_pres)
pp_pres = pp_pres[pp_pres.between(pp_pres.quantile(.15), pp_pres.quantile(.85))]
pp_pres = mean(pp_pres)

avg_sys_rise = pd.Series(avg_sys_rise)
avg_sys_rise = avg_sys_rise[avg_sys_rise.between(avg_sys_rise.quantile(.15), avg_sys_rise.quantile(.85))]
avg_sys_rise = mean(avg_sys_rise)

sys_rise_area = pd.Series(sys_rise_area)
sys_rise_area = sys_rise_area[sys_rise_area.between(sys_rise_area.quantile(.15), sys_rise_area.quantile(.85))]
sys_rise_area = mean(sys_rise_area)

t_sys_rise = pd.Series(t_sys_rise)
t_sys_rise = t_sys_rise[t_sys_rise.between(t_sys_rise.quantile(.15), t_sys_rise.quantile(.85))]
t_sys_rise = mean(t_sys_rise)

avg_dec = pd.Series(avg_dec)
avg_dec = avg_dec[avg_dec.between(avg_dec.quantile(.15), avg_dec.quantile(.85))]
avg_dec = mean(avg_dec)

t_dec = pd.Series(t_dec)
t_dec = t_dec[t_dec.between(t_dec.quantile(.15), t_dec.quantile(.85))]
t_dec = mean(t_dec)

dec_area = pd.Series(dec_area)
dec_area = dec_area[dec_area.between(dec_area.quantile(.15), dec_area.quantile(.85))]
dec_area = mean(dec_area)

avg_sys = pd.Series(avg_sys)
avg_sys = avg_sys[avg_sys.between(avg_sys.quantile(.15), avg_sys.quantile(.85))]
avg_sys = mean(avg_sys)

slope_sys = pd.Series(slope_sys)
slope_sys = slope_sys[slope_sys.between(slope_sys.quantile(.15), slope_sys.quantile(.85))]
slope_sys = mean(slope_sys)

sys_area = pd.Series(sys_area)
sys_area = sys_area[sys_area.between(sys_area.quantile(.15), sys_area.quantile(.85))]
sys_area = mean(sys_area)

t_sys = pd.Series(t_sys)
t_sys = t_sys[t_sys.between(t_sys.quantile(.15), t_sys.quantile(.85))]
t_sys = mean(t_sys)

avg_sys_dec = pd.Series(avg_sys_dec)
avg_sys_dec = avg_sys_dec[avg_sys_dec.between(avg_sys_dec.quantile(.15), avg_sys_dec.quantile(.85))]
avg_sys_dec = mean(avg_sys_dec)

dn_sys = pd.Series(dn_sys)
dn_sys = dn_sys[dn_sys.between(dn_sys.quantile(.15), dn_sys.quantile(.85))]
dn_sys = mean(dn_sys)

sys_dec_area = pd.Series(sys_dec_area)
sys_dec_area = sys_dec_area[sys_dec_area.between(sys_dec_area.quantile(.15), sys_dec_area.quantile(.85))]
sys_dec_area = mean(sys_dec_area)

t_sys_dec = pd.Series(t_sys_dec)
t_sys_dec = t_sys_dec[t_sys_dec.between(t_sys_dec.quantile(.15), t_sys_dec.quantile(.85))]
t_sys_dec = mean(t_sys_dec)

avg_sys_dec_nodia = pd.Series(avg_sys_dec_nodia)
avg_sys_dec_nodia = avg_sys_dec_nodia[avg_sys_dec_nodia.between(avg_sys_dec_nodia.quantile(.15), avg_sys_dec_nodia.quantile(.85))]
avg_sys_dec_nodia = mean(avg_sys_dec_nodia)

avg_sys_nodia = pd.Series(avg_sys_nodia)
avg_sys_nodia = avg_sys_nodia[avg_sys_nodia.between(avg_sys_nodia.quantile(.15), avg_sys_nodia.quantile(.85))]
avg_sys_nodia = mean(avg_sys_nodia)

avg_sys_rise_nodia = pd.Series(avg_sys_rise_nodia)
avg_sys_rise_nodia = avg_sys_rise_nodia[avg_sys_rise_nodia.between(avg_sys_rise_nodia.quantile(.15), avg_sys_rise_nodia.quantile(.85))]
avg_sys_rise_nodia = mean(avg_sys_rise_nodia)

avg_dec_nodia = pd.Series(avg_dec_nodia)
avg_dec_nodia = avg_dec_nodia[avg_dec_nodia.between(avg_dec_nodia.quantile(.15), avg_dec_nodia.quantile(.85))]
avg_dec_nodia = mean(avg_dec_nodia)
             
slope_dia = pd.Series(slope_dia)
slope_dia = slope_dia[slope_dia.between(slope_dia.quantile(.15), slope_dia.quantile(.85))]
slope_dia = mean(slope_dia)

t_dia = pd.Series(t_dia)
t_dia = t_dia[t_dia.between(t_dia.quantile(.15), t_dia.quantile(.85))]
t_dia = mean(t_dia)

avg_dia = pd.Series(avg_dia)
avg_dia = avg_dia[avg_dia.between(avg_dia.quantile(.15), avg_dia.quantile(.85))]
avg_dia = mean(avg_dia)

dn_dia = pd.Series(dn_dia)
dn_dia = dn_dia[dn_dia.between(dn_dia.quantile(.15), dn_dia.quantile(.85))]
dn_dia = mean(dn_dia)